In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load a text file (any book / text)
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Character vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Character ↔ integer mapping
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

# Encode entire dataset as integers
data = torch.tensor([stoi[c] for c in text], dtype=torch.long)

In [15]:
cpu_only=True
device = "cpu" if cpu_only else "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.set_default_dtype(torch.float32)
print("device:",device)

device: cpu


In [16]:
# Example usage of view and transpose
test=torch.arange(6,device=device)
print(test)
test = test.view(2,3)
print(test)
test = test.transpose(0,1)
print(test)

tensor([0, 1, 2, 3, 4, 5])
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


In [17]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 64   # context length
batch_size = 32

def get_batch(split):
    data_src = train_data if split == "train" else val_data
    ix = torch.randint(len(data_src) - block_size -1, (batch_size,))

    # Input tokens
    x = torch.stack([data_src[i:i+block_size] for i in ix])
    # Target = next character
    y = torch.stack([data_src[i+1:i+block_size+1] for i in ix])

    return x.to(device), y.to(device)

In [19]:
get_batch("train")

(tensor([[ 6,  0, 32,  ..., 39, 58,  1],
         [53, 58,  1,  ..., 57,  1, 44],
         [39, 58, 58,  ..., 43, 12,  0],
         ...,
         [60, 47, 53,  ..., 24, 13, 26],
         [43, 56, 43,  ..., 46, 63,  1],
         [41, 41, 59,  ..., 61, 43, 39]]),
 tensor([[ 0, 32, 46,  ..., 58,  1, 51],
         [58,  1, 40,  ...,  1, 44, 39],
         [58, 58, 43,  ..., 12,  0, 16],
         ...,
         [47, 53, 50,  ..., 13, 26, 33],
         [56, 43, 11,  ..., 63,  1, 61],
         [41, 59, 57,  ..., 43, 39, 49]]))

In [20]:
class CharEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, block_size):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size, d_model)

    def forward(self, x):
        B, T = x.shape
        tok = self.token_emb(x)              # (B, T, d_model)
        pos = self.pos_emb(torch.arange(T, device=device))  # (T, d_model)
        return tok + pos


In [21]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, block_size):
        super().__init__()
        assert d_model % n_heads == 0

        self.n_heads = n_heads
        self.d_head = d_model // n_heads

        self.key   = nn.Linear(d_model, d_model, bias=False)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)

        self.proj = nn.Linear(d_model, d_model)

        self.register_buffer(
            "mask",
            torch.tril(torch.ones(block_size, block_size)).bool()
        )

    def forward(self, x):
        B, T, C = x.shape

        # Project once
        K = self.key(x)    # (B, T, C)
        Q = self.query(x)
        V = self.value(x)

        # Split into heads
        K = K.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        Q = Q.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        V = V.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        # Shapes: (B, n_heads, T, d_head)

        # Attention scores
        att = (Q @ K.transpose(-2, -1)) / (self.d_head ** 0.5)
        # (B, n_heads, T, T)

        att = att.masked_fill(~self.mask[:T, :T], float('-inf'))
        att = F.softmax(att, dim=-1)

        # Weighted sum
        out = att @ V  # (B, n_heads, T, d_head)

        # Recombine heads
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        return self.proj(out)

In [22]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, block_size, head_n):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttention(d_model, head_n, block_size)
        self.ln2 = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )

    def forward(self, x):
        # Attention with residual
        x = x + self.attn(self.ln1(x))
        # Feed-forward with residual
        x = x + self.ff(self.ln2(x))
        return x

In [23]:
class CharLM(nn.Module):
    def __init__(self, vocab_size, d_model, block_size,head_n):
        super().__init__()
        self.embed = CharEmbedding(vocab_size, d_model, block_size)
        self.block = TransformerBlock(d_model,block_size, head_n)
        self.ln = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):
        x = self.embed(x)        # (B, T, d_model)
        x = self.block(x)
        x = self.ln(x)
        logits = self.head(x)   # (B, T, vocab_size)

        if targets is None:
            return logits

        B, T, V = logits.shape
        probs=logits.view(B*T, V) # probabilities: B * T, V
        ids = targets.view(B*T) # ids: B * T
        loss = F.cross_entropy(
            probs,
            ids
        )
        return logits, loss

In [24]:
model = CharLM(vocab_size, d_model=128, block_size=block_size, head_n=4).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for step in range(5000):
    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 500 == 0:
        print(f"step {step}, loss {loss.item():.4f}")


step 0, loss 4.3923
step 500, loss 2.4035
step 1000, loss 2.1853
step 1500, loss 2.0028
step 2000, loss 1.9638
step 2500, loss 1.9110
step 3000, loss 1.8846
step 3500, loss 1.7913
step 4000, loss 1.7194
step 4500, loss 1.6580


In [25]:
@torch.no_grad()
def generate(model, start, max_new_tokens=200):
    model.eval()
    idx = torch.tensor([[stoi[c] for c in start]], device=device)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = model(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)

    return "".join(itos[i.item()] for i in idx[0])


In [26]:
print(generate(model, "CORIOLANUS:"))

CORIOLANUS:
Which have spare of Julike himssianks
To spee him
Youcking, I sweep, appofor celf hear of and have be do thusbost,
And man: lodden than all it otruember
Tillies too was what kneethes
houl be an are f
